# Gallery Example: Er/M/1 Reentrant Queue

This example demonstrates an Erlang/M/1 queueing system with reentrant routing:
- **Arrivals**: Erlang distribution
- **Service**: Exponential service times
- **Servers**: 1 server
- **Scheduling**: FCFS
- **Reentrant**: Jobs switch class and pass through the queue again

In reentrant queues, jobs complete service and re-enter the queue as a different class.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_erlm1_reentrant(): Network {
    """Create Er/M/1-Reentrant queueing model"""
    val model = Network("Er/M/1-Reentrant")
// Block 1: nodes
    val source = Source(model, "Source")
    val queue = Queue(model, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model, "Sink")
// Block 2: classes
    val oclass1 = OpenClass(model, "Class1")
    val oclass2 = OpenClass(model, "Class2")
// Erlang arrival for Class1, Class2 is disabled at source
    source.setArrival(oclass1, Erlang.fit_mean_and_order(1, 5))
    source.setArrival(oclass2, Disabled())
// Different service rates for each class
    queue.setService(oclass1, Exp(2))
    queue.setService(oclass2, Exp(3))
// Block 3: topology with class switching (reentrant)
    val P = model.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)        # Arrivals go to queue as Class1
    P.addClassSwitch(oclass1, oclass2, queue, queue, 1.0)  # Class1 switches to Class2 (reentrant)
    P.addRoute(oclass2, queue, sink, 1.0)          # Class2 exits to sink
    model.link(P)
    
    return model
// Create the model
val model = gallery_erlm1_reentrant()

## About Reentrant Queues

In reentrant queueing systems:
- Jobs pass through the same server multiple times
- Each pass may have different service characteristics (class switching)
- Models manufacturing systems with rework or multi-stage processing

In this model:
1. Jobs arrive as Class1 with Erlang(1, 5) inter-arrival times
2. After service (Exp(2)), they become Class2
3. Class2 jobs receive service (Exp(3)) and then exit

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)